In [3]:
# %load_ext sage
import os
os.chdir(os.path.expanduser("~/Dropbox/ABACUS/Workspace/LeapArticulatorQt"))
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
# from ExperimentalData import fromFile, HMM, ViterbiPath, levenshtein
from leaparticulator.data.functions import fromFile
from leaparticulator.data.hmm import HMM, ViterbiPath, levenshtein

!cd ~/Dropbox/ABACUS/Workspace/LeapArticulatorQt
files = !ls img/meanings/*_*.png
# print files
images_phase1 = !ls img/meanings/5_*.png
images_phase2 = !ls img/meanings/*_[135].png
images_phase3 = images_phase2
print images_phase1

def title(m):
    return m.split("/")[-1]

def titles(file_list):
    return [title(f) for f in file_list]

def meaning_to_coordinates(meaning):
    return tuple(map(float,meaning.split("/")[-1].split(".")[0].split("_")))

def distance(m1, m2):
#     m1_1, m1_2 = m1.split("/")[-1].split(".")[0].split("_")
#     m2_1, m2_2 = m2.split("/")[-1].split(".")[0].split("_")
#     m1_1 = float(m1_1)
#     m1_2 = float(m1_2)
#     m2_1 = float(m2_1)
#     m2_2 = float(m2_2)
    return euclidean(meaning_to_coordinates(m1), meaning_to_coordinates(m2))#sqrt((m2_1-m1_1)**2 + (m2_2-m1_2)**2)

def distance_matrix(meanings, labels, normalize=False):
#     matrix = np.asarray([[distance(m1, m2) for m1 in meanings] for m2 in meanings])
    matrix = np.asarray([[distance(m1, m2) for m1 in meanings] for m2 in meanings])
    matrix = pd.DataFrame(matrix, index=titles(labels), columns=titles(labels))
    if normalize:
        maximum = np.max(matrix)
        matrix = np.divide(matrix, maximum)
#     print meanings
#     print matrix
    return matrix

def viterbi_path(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}
 
    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]
 
    # Run Viterbi for t > 0
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
 
        for y in states:
            (prob, state) = max((V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]], y0) for y0 in states)
            V[t][y] = prob
            newpath[y] = path[state] + [y]
 
        # Don't need to remember the old paths
        path = newpath
    n = 0           # if only one element is observed max is sought in the initialization values
    if len(obs)!=1:
        n = t
    print_dptable(V)
    (prob, state) = max((V[n][y], y) for y in states)
    return (prob, path[state])

def pick_lowest_bic(hmms):
    hmm, bic = None, 99999999999999
    for h in hmms:
        if h is None:
            continue
        if h.bic < bic:
            hmm = h
            bic = h.bic
    if hmm == None:
        print f, hmms
    return (hmm, bic)

def hmm_to_sage_hmm(hmm):
    transmat = hmm.transmat
    initProb = hmm.initProb
    new = hmm.GaussianHiddenMarkovModel()

# meaning_matrix = np.asarray([[distance(f, f2) for f2 in files] for f in files])
# meaning_matrix = pd.DataFrame(meaning_matrix, index=[title(f) for f in files], columns=[title(f) for f in files])e

['img/meanings/5_1.png', 'img/meanings/5_2.png', 'img/meanings/5_3.png', 'img/meanings/5_4.png', 'img/meanings/5_5.png']


In [4]:
# %load_ext rpy2.ipython
files = !ls ~/ThereminData/logs/discrete/D*phase*.*
blacklist = ['123R0126514']
files = [".".join((f.split('.')[:-1])) for f in files]
files = filter(lambda x: x.split(".")[-1] in ('xy', 'amp_and_freq', 'amp_and_mel'), files)
files = filter(lambda x: not any([el in x for el in blacklist]), files)
files = map(os.path.abspath, files)
file_to_id = lambda f: ".".join(f.split('/')[-1].split('.')[:2])
files_xy = filter(lambda x: x.split(".")[-1] in ('xy',), files)
files_amp_and_mel = filter(lambda x: x.split(".")[-1] in ('amp_and_mel',), files)
files_amp_and_freq = filter(lambda x: x.split(".")[-1] in ('amp_and_freq',), files)
# for f in files:
#     print f
# print files
ids = [file_to_id(f) for f in files]
# print ids
print len(files), "files"
print files_xy[0], files_amp_and_mel[0], files_amp_and_freq[0] 
print set(ids), len(set(ids))

270 files
/home/kerem/ThereminData/logs/discrete/D12300130914.1.exp.log.phase0.xy /home/kerem/ThereminData/logs/discrete/D12300130914.1.exp.log.phase0.amp_and_mel /home/kerem/ThereminData/logs/discrete/D12300130914.1.exp.log.phase0.amp_and_freq
set(['D123009131014.1', 'D13200591014.2', 'D123015171014.1', 'D132007131014.2', 'D132009141014.2', 'D12300521014.1', 'D123013151014.1', 'DR123012151014.1r', 'D13200111014.2', 'DR132008131014.2r', 'DR123008101014.1r', 'D132011151014.2', 'DR12300230914.1r', 'DR13200421014.2r', 'D12300130914.1', 'D123011151014.1', 'DR132010141014.2r', 'D123007101014.1', 'DR132012151014.2r', 'DR123014161014.1r', 'DR12300691014.1r', 'DR12300411014.1r', 'DR123010141014.1r', 'DR132014161014.2r', 'DR123016161014.1r', 'DR13200211014.2r', 'D12300311014.1', 'D13200321014.2', 'D132013161014.2', 'DR132006101014.2r']) 30


In [5]:
from rpy2.robjects import r, globalenv, FloatVector, ListVector

def is_multivariate(hmm):
    return hmm.multivariate
#     try:
# #         print hmm.means
#         iter(hmm.means[0])
#         return True
#     except:
#         return False
    
def r_list_of_stuff(arr, func, name):
    r("%s <- list()" % name)
    for i, v in enumerate(arr):
        globalenv["tmp"] = func(v)
        r("%s[[%d]] <- tmp" % (name, i + 1))
    del globalenv["tmp"]
    
def rhmm_dist_from_hmm(hmm):
    print "rhmm_dist_from_hmm"
    mean = np.asarray(hmm.means)
    cov = np.asarray(hmm.variances)
#     print cov

    command = "dist <- distributionSet(dis='NORMAL', mean=%s, cov=%s)"
  
    globalenv['mean'] = mean 
    globalenv['covar'] = cov
    
    if is_multivariate(hmm):
        f = lambda x: r.matrix(FloatVector(np.asarray(x).flatten()), nrow=len(x))
        r_list_of_stuff(hmm.variances, func=f, name="covar")
        f = lambda x: FloatVector(np.asarray(x))
        r_list_of_stuff(hmm.means, func=f, name="mean")
#         r("print(covar)")
#         r("print(mean)")
        command = "dist <- distributionSet(dis='NORMAL', mean=mean, cov=covar)"
    else:
        command = "dist <- distributionSet(dis='NORMAL', mean=as.vector(mean), var=as.vector(covar))"
#     print command
    r(command)
    return globalenv['dist']

def rhmm_from_hmm(hmm):
    print "rhmm_from_hmm"
    from rpy2.robjects import r, globalenv
    r("rm(list=ls())")
    r("source('SampleHMM.R');library('RHmm')")
    hmm_ = hmm
    dist = rhmm_dist_from_hmm(hmm_)
    globalenv['initProb'] = np.asarray(hmm_.initProb)
    globalenv['transMat'] = np.asarray(hmm_.transmat)
    command = "hmm <- HMMSet(initProb=as.vector(initProb), transMat=as.matrix(transMat), distribution=dist)"
#     print command
    r(command)
#     print r("print(hmm)")
    return globalenv['hmm']
    
def test_rhmm_from_hmm_univariate():
    prep=   """
            source("SampleHMM.R");
            list[hmm, d] <- fitHMMtoPhase(file_id="1230105514.master", nStates=9, iter=1000, phase=0, take_vars=c(%s));
            """
    r(prep % ("'x'"))
    hmm = globalenv['hmm']
    a = rhmm_from_hmm(HMM(hmm, [1,2,3,4]))
    assert str(hmm[0]) == str(a)
    del globalenv['hmm']
    
def test_rhmm_from_hmm_multivariate():
    prep=   """
            source("SampleHMM.R");
            list[hmm, d] <- fitHMMtoPhase(file_id="1230105514.master", nStates=9, iter=1000, phase=1, take_vars=c(%s));
            """
    r(prep % ("'x','y'"))
    hmm = globalenv['hmm']
    a = rhmm_from_hmm(HMM(hmm, [1,2,3,4]))
    assert str(hmm[0]) == str(a)
    del globalenv['hmm']
    

In [6]:
import os; print os.getcwd()
from leaparticulator.constants import palmToAmpAndFreq,palmToAmpAndMel
import leaparticulator.constants as Constants

def isMultivariate(cond, phase):
    phase = int(phase)
    if cond in ("2","2r"):
        if phase == 1:
            return True
    else:
        if phase == 2:
            return True
    return False

def formatTrajectory(trajectory, responses, responses_p, phase, cond, units=Constants.AMP_AND_FREQ):    
    multivariate = False
    reverse_cond = cond in ("2r","1r")
    interval = 1
    pick_var = 0
    phase = int(phase)
    if reverse_cond:
        interval = -1
        pick_var = 1
    
    multivariate = isMultivariate(cond, phase)
            
    formatData = None
    print "Multivariate?", multivariate
    
    if multivariate:
        if units == Constants.XY:
            formatData = lambda r, phase: [[frame.get_stabilized_position()[:2][::interval] for frame in rr] for rr in r["127.0.0.1"][str(phase)].values()]
        elif units == Constants.AMP_AND_FREQ:
            # -interval, because amp_and_freq returns y,x and not x,y. 
            formatData = lambda r, phase: [palmToAmpAndFreq(frame.get_stabilized_position())[::-interval] for frame in r]
        elif units == Constants.AMP_AND_MEL:
            # -interval, because amp_and_freq returns y,x and not x,y. 
            formatData = lambda r, phase: [[palmToAmpAndMel(frame.get_stabilized_position())[::-interval] for frame in rr] for rr in r["127.0.0.1"][str(phase)].values()]
    else:
        if units == Constants.XY:
            formatData = lambda r, phase: [[frame.get_stabilized_position()[pick_var] for frame in rr] for rr in r["127.0.0.1"][str(phase)].values()]
        elif units == Constants.AMP_AND_FREQ:
            # -interval, because amp_and_freq returns y,x and not x,y. 
            formatData = lambda r, phase: [[palmToAmpAndFreq(frame.get_stabilized_position())[::-interval][pick_var] for frame in r]]#r] for rr in r["127.0.0.1"][str(phase)].values()]
        elif units == Constants.AMP_AND_MEL:
            # -interval, because amp_and_freq returns y,x and not x,y. 
            formatData = lambda r, phase: [[palmToAmpAndMel(frame.get_stabilized_position())[::-interval][pick_var] for frame in rr] for rr in r["127.0.0.1"][str(phase)].values()]
    
    data = formatData(trajectory,phase)
#     print "len(data)=", len(data[0])
#     data = zip(data[0], data[1], data[2]) # + formatData(responses_p,phase)
    if not multivariate:
        return data[0]
    else:
        return data

/shared/Dropbox/ABACUS/Workspace/LeapArticulatorQt


In [7]:
print os.getcwd()

/shared/Dropbox/ABACUS/Workspace/LeapArticulatorQt


In [8]:
import warnings
warnings.filterwarnings('ignore')
from leaparticulator.data.functions import stderr_redirected
from leaparticulator.notebooks.StreamlinedDataAnalysisGhmm import unpickle_results
from itertools import product
import skbio
from skbio.stats.distance import mantel, DistanceMatrix
import jsonpickle
import pickle
# from BICComparison import pick_lowest_bic

ids = list(set(ids))
to_log = lambda x: os.path.join(os.path.expanduser("~/ThereminData"), "logs/discrete/logs/" + x + ".exp.log")
to_hmm = lambda x: os.path.join(os.path.expanduser("~/ThereminData"), "logs/discrete/" + x + ".exp.log")
to_hmm_file = lambda id, phase, units: to_hmm(id) + ".phase" + phase + "." + units + ".hmms"
to_icon_file = lambda x: to_hmm(x) + ".phase%s" % phase + ".iconicity"
hmm = None


# for each participant
for id in ids:
# for f in files:
    print id
    f = to_log(id)
    # determine the experimental condition
    cond = id.split(".")[1]
    units = Constants.AMP_AND_FREQ
    if cond == "master":
        cond = "1" 
    assert cond in ("1","2","1r","2r")
    
    responses_, tests, responses_t_, tests_t, images = fromFile(f)
    responses, tests, responses_t, tests_t, images = responses_['127.0.0.1'], \
                                                     tests['127.0.0.1'], \
                                                     responses_t_['127.0.0.1'], \
                                                     tests_t['127.0.0.1'], images \
    # for each phase
    for phase in responses.keys():
#         if not isMultivariate(cond, phase):
#             print "NOT MULTIVARIATE"
#             continue
        print "Phase/cond/r: %s/%s/%s" % (phase, cond, cond in ("2r","1r")) 
        # find the relevant images
        image_list = images_phase1
        if phase == "1":
            image_list = images_phase2
        elif phase == "2":
            image_list = images_phase3
        else:
            if phase != "0":
                print "What the fuck is a phase %s?!?" % phase

        # compute the meaning distance matrix
        meaning_matrix = distance_matrix(image_list, image_list, normalize=False)

#         trajectory_matrix_by_prob = meaning_matrix.copy()
        trajectory_matrix_by_stateseq = meaning_matrix.copy()
        viterbis = {}
        print "Images for the phase are: %s" % map(title, responses[phase].keys())
        # for each image
        for image in responses[phase].keys():
            # find and fetch the relevant hmm
            print title(image)
            hmm_file = to_hmm_file(id, phase, "amp_and_freq")
            results = unpickle_results(hmm_file)
            hmm, bic = pick_lowest_bic(results.hmms)
            # ...and send it to R
#             rhmm_from_hmm(hmm)
            
            # find the relevant trajectory
            trajectory = responses[phase][image]
            print trajectory[:15]
            command = None
            trajectory = formatTrajectory(trajectory, responses_, responses_t_, phase, cond)
#             print trajectory[0]
#             print "Is multivariate?", is_multivariate(hmm)
#             if is_multivariate(hmm):
#                 trajectory = [frame.get_stabilized_position()[:2] for frame in trajectory]   
# #                 trajectory = [FloatVector(frame.get_stabilized_position()[:2]) for frame in trajectory]   
# #                 globalenv['trajectory'] = ListVector(trajectory)
#             else:
#                 trajectory = [frame.get_stabilized_position()[0] for frame in trajectory]   
#                 trajectory = [frame.get_stabilized_position()[0] for frame in trajectory]   
#                 globalenv['trajectory'] = FloatVector(trajectory)
#             print trajectory
            # compute the corresponding state sequence            
#             globalenv['trajectory'] = ListVector(trajectory)
#             print trajectory
#             command = "viterbi(hmm, trajectory)"
            
#             viterbi = r(command)
#             print pd.DataFrame(hmm.means)
#             print np.asarray(hmm.means).shape
#             print np.asarray(hmm.means).shape[1]
#             with SilentGhmmContext(stdout=sys.stdout):
#             with stderr_redirected():
            print "Raw trajectory sample (%s):" % len(trajectory), trajectory[:15]
            if not isMultivariate(cond, int(phase)):
                trajectory = [trajectory]
            result, likelihood = hmm.viterbi(trajectory)
#             print result
            import sys;sys.stdout.flush()
#             print result
            viterbis[title(image)] = result
            if len(result) == 0:
                print "Image %s at phase %s didn't yield a viterbi path (id: %s)" % (title(image), phase, id)
#             print ViterbiPath(viterbi)
        images = titles(responses[phase])
        maximum = 0
        for image1, image2 in product(images,images):
#             print image1, image2
            dist = levenshtein(viterbis[image1], viterbis[image2])
            trajectory_matrix_by_stateseq[image1][image2] = dist
            maximum = max((maximum, dist))
#         print trajectory_matrix_by_stateseq, maximum
        trajectory_matrix_by_stateseq = np.divide(trajectory_matrix_by_stateseq, maximum)
#         break
#         print "Meaning matrix:"
#         print meaning_matrix
#         print "Signal matrix:"
#         print trajectory_matrix_by_stateseq
#         print meaning_matrix.T == meaning_matrix
#         print trajectory_matrix_by_stateseq.T == trajectory_matrix_by_stateseq
        d1 = DistanceMatrix(meaning_matrix)
        d2 = DistanceMatrix(trajectory_matrix_by_stateseq)
        coef = mantel(d1, d2, method="spearman")
        print d1
        print d2
        print "Correlation coefficient:",coef[0], "(p=%f)" % coef[1]
        icon_file = to_icon_file(id)
        print "Writing to file: %s" % icon_file
        with open(icon_file, "wb") as f:
            pickle.dump((d1,d2,coef), f)
#             item = str((d1, d2, coef)).decode('cp1252').encode("utf-8")
#             f.write(jsonpickle.encode((d1, d2, coef)))
        import sys;sys.stdout.flush()
#         break
#         from time import sleep;sleep(1)

/shared/Dropbox/ABACUS/Workspace/LeapArticulatorQt
D123009131014.1
Phase/cond/r: 1/1/False
Images for the phase are: ['1_1.png', '5_5.png', '1_3.png', '5_1.png', '5_3.png', '3_5.png', '3_3.png', '3_1.png', '1_5.png']
1_1.png
[<leaparticulator.data.frame.LeapFrame object at 0x7fa32af3a8d0>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3a990>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3aa50>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3ab10>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3abd0>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3ac90>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3ad50>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3ae10>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3aed0>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af3af90>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af43090>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32af43

GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model



Multivariate? True
Raw trajectory sample (604): [(0.06706881135428522, 307.2406633967109), (0.06650606974612328, 307.7139832515137), (0.0659783410430137, 308.1670222670446), (0.06547847298360732, 308.6053728845966), (0.06496403779980087, 309.06256569328406), (0.06445007861731833, 309.50624034477596), (0.06399816640333356, 309.90730269411904), (0.06355764746612458, 310.2889297483306), (0.06318253961463571, 310.67199347077906), (0.06284616376268382, 311.03068621431356), (0.06251551634593189, 311.3682630492597), (0.06220020273826199, 311.6916411316297), (0.061901774200336046, 312.02283308842635), (0.06158951142549762, 312.34723460265604), (0.061292461577651336, 312.6529081059888)]
3_1.png
[<leaparticulator.data.frame.LeapFrame object at 0x7fa32a958490>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32a958550>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32a958610>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32a9586d0>, <leaparticulator.data.frame.LeapFrame objec

GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model



Multivariate? True
Raw trajectory sample (352): [(0.15779559809920707, 110.97277352617144), (0.15508333888719317, 111.69679730831864), (0.15270974066369092, 112.42058835897683), (0.15007696797987513, 113.3342131914865), (0.14741454791678732, 114.33613706595695), (0.14512977763822132, 115.28394799630509), (0.14295182774530102, 116.50760113418447), (0.14064646514346002, 117.97700614932539), (0.13917879848354897, 118.83489487712977), (0.13814515373539626, 119.54337291284939), (0.13703298490406046, 120.36933163271996), (0.13616918541667, 121.03914418181442), (0.13543454352216655, 121.64548389427078), (0.13486490622249148, 122.39470104898264), (0.13427541493969686, 123.26831571934416)]
5_1.png
[<leaparticulator.data.frame.LeapFrame object at 0x7fa32574f610>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32574f710>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32574f810>, <leaparticulator.data.frame.LeapFrame object at 0x7fa32574f910>, <leaparticulator.data.frame.LeapFrame ob

...


Multivariate? True
Raw trajectory sample (152): [(0.05437741156051912, 480.7072430655292), (0.05200975655012097, 476.99011707256), (0.05005174645363186, 472.01159137373594), (0.043703492947209766, 465.3000517671772), (0.04164696982594873, 460.79633953845456), (0.0405027832189111, 457.1716038341759), (0.043114111883766126, 457.8650990413512), (0.04000187224127383, 450.7506712964876), (0.03798381969226816, 445.65185729619424), (0.0387170968830064, 442.5587689463815), (0.03902755326892504, 439.4388489978159), (0.038702848857757655, 435.16915599833084), (0.03685941429758932, 429.80394565406976), (0.03514418122874474, 424.8426358910403), (0.03352403913351254, 420.06099742051686)]
9x9 distance matrix
IDs:
'0', '1', '2', '3', '4', '5', '6', '7', '8'
Data:
[[ 0.          2.          4.          2.          2.82842712  4.47213595
   4.          4.47213595  5.65685425]
 [ 2.          0.          2.          2.82842712  2.          2.82842712
   4.47213595  4.          4.47213595]
 [ 4.         

GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viter


Multivariate? True
Raw trajectory sample (292): [(1703.6771691237432, 0.09980994595709092), (1703.7100129580092, 0.09980120044960272), (1703.3173564093977, 0.09981118272960665), (1703.182303477539, 0.0997991024739957), (1703.157749549502, 0.09977945997610793), (1703.5740721573425, 0.0997258210835732), (1703.606914004084, 0.0996999401218881), (1703.4666980165507, 0.0996884411115464), (1703.3970226737538, 0.09967976767699893), (1703.7439998530988, 0.09964517738468781), (1703.287946507956, 0.09965000969838211), (1703.8953782925569, 0.0995746049933508), (1705.219227153096, 0.0994600909334844), (1706.5469653501573, 0.09936559723209104), (1707.8190509801173, 0.0992910926011974)]
3_3.png
[<leaparticulator.data.frame.LeapFrame object at 0x7fa3263f9390>, <leaparticulator.data.frame.LeapFrame object at 0x7fa3263f9450>, <leaparticulator.data.frame.LeapFrame object at 0x7fa3263f9510>, <leaparticulator.data.frame.LeapFrame object at 0x7fa3263f95d0>, <leaparticulator.data.frame.LeapFrame object at 

GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
GHMM ghmm.py:148 - sviterbi.c:ghmm_cmodel_viterbi(215): sequence can't be build from model
ERROR:GHMM:sviterbi.c:ghmm_cmodel_viter


Multivariate? True
Raw trajectory sample (315): [(735.0334345115184, 0.15289716212429882), (735.0108860952101, 0.1528488640912622), (734.9868598554614, 0.15279941006966524), (734.9526084203297, 0.15274619771067888), (734.9057924939311, 0.15268711584235928), (734.854175232016, 0.15262630900677143), (734.79344643734, 0.15256377894731765), (734.7209603194142, 0.15249716407337643), (734.6426316198786, 0.15242800449401306), (734.5546449514801, 0.15235863515786652), (734.4510940190651, 0.15229036943370444), (734.3313083601566, 0.15222118552134278), (734.187480589437, 0.15214946277706887), (734.0209783638733, 0.15207689895120247), (733.8331702992859, 0.15200415795445288)]
3_1.png
[<leaparticulator.data.frame.LeapFrame object at 0x7fa3267bd2d0>, <leaparticulator.data.frame.LeapFrame object at 0x7fa3267bd390>, <leaparticulator.data.frame.LeapFrame object at 0x7fa3267bd450>, <leaparticulator.data.frame.LeapFrame object at 0x7fa3267bd510>, <leaparticulator.data.frame.LeapFrame object at 0x7fa326

In [ ]:
ids[-1]

In [ ]:
%load_ext sage
from random import random
states = 3
transmat = np.asarray([[.2,.3,.5],[.3, .6, .1], [.5, .3, .2]])
initProb = np.asarray([.3,.3,.4])
means = np.asarray([(i,i) for i in range(states)])
spreads = np.asarray([[[i,i],[i,i]] for i in range(states)])
print "Transmat", transmat
print "Init", initProb
print "Mean", means
print "Spread", spreads
x = [[mean, spread] for mean, spread in zip(means, spreads)]
print x
hmm.GaussianHiddenMarkovModel(transmat, x, initProb)

In [ ]:
from LeapTheremin import palmToAmpAndFreq, freqToMel
import numpy as np

In [ ]:
grid_x = np.linspace(-250,250,30)
grid_y = np.linspace(1,500,30)
grid_z = grid_y

In [ ]:
from itertools import product
f = figure()
x, y, z = zip(*[i for i in product(grid_x, grid_y, grid_z)])
scatter(x,y,s=2,color='blue')
f.suptitle('XY Projection')
figure().suptitle('Amplitude/Frequency Projection')
y, x = zip(*[palmToAmpAndFreq(i) for i in product(grid_x, grid_y, grid_z)])
scatter(x,y,s=2,color='blue')

figure().suptitle('Amplitude/MelFrequency Projection')
x = [freqToMel(xx) for xx in x]
scatter(x,y,s=2,color='blue')

In [ ]:
%matplotlib qt
from matplotlib import pylab
from LeapTheremin import palmToAmpAndFreq
pylab.figure()
id = "132R0128514.2r"
responses, tests, responses_t, tests_t, images = fromFile(to_log(id))
responses = responses['127.0.0.1']
hmm_file = to_hmm_file(id, phase, "amp_and_freq")
results = unpickle_results(hmm_file)
hmm, bic = pick_lowest_bic(results.hmms)
print hmm.means
reload(HMMPlotting)
HMMPlotting.plot_hmm(hmm.means, hmm.transmat, hmm.variances)
# data = [f.get_stabilized_position() for f in responses['1']["./img/meanings/3_5.png"]]
data = HMMPlotting.to_trajectory_object([v for v in responses['1'].values()], units="amp_and_freq")
# print data
HMMPlotting.plot_trajectories(data)
pylab.show()

In [ ]:
results = unpickle_results("logs/ghmm/132R0128514.2r.exp.log.phase1")

In [ ]:
responses, test_results, responses_practice, test_results_practice, images = fromFile("logs/132R0128514.2r.exp.log")

In [ ]:
import pandas as pd
len(responses['127.0.0.1']['1']['./img/meanings/1_1.png'])